In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk

from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: license expired

    the package mklfft 2.1 was published on 2016-02-09,
    however the mkl license expiration date is 2015-11-08.
    You may be able to run earlier versions of mklfft using your
    current license.  A new license can be purchased at: http://continuum.io
    To revert to an earlier set of conda packages, use:
    $ conda list --revisions
    ...
    $ conda install --revision <REVISION NUMBER>

    
/Users/dtraverso/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
df_mp = pd.read_json(path_or_buf="file://localhost/Users/dtraverso/projects/nasa_nea/data/mpcorb_extended.json")

In [3]:
df_mp.head()

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Ref,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms
0,2.977808,NaN,1801-2015,MPCLINUX,NaN,2457400.5,0.12,3.34,0000,2015-11-28,...,MPO370963,1.376113,1.277355,2.458235e+06,0,2.768112,0.075754,10.59166,0.214007,0.60
1,3.412843,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.11,4.13,0000,2015-10-16,...,MPO350795,1.312198,1.276541,2.456634e+06,0,2.772362,0.231024,34.84095,0.213515,0.58
2,3.353932,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.32,5.33,0000,2015-12-18,...,MPO370963,1.247217,1.297421,2.456854e+06,0,2.669763,0.256266,12.98987,0.225941,0.60
3,2.571675,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.32,3.20,0000,2015-12-15,...,MPO370963,1.171498,1.380322,2.456924e+06,0,2.361670,0.088922,7.14005,0.271566,0.60
4,3.066885,NaN,1845-2015,MPCLINUX,NaN,2457400.5,0.15,6.85,0000,2015-12-20,...,MPO370963,1.239938,1.319481,2.457419e+06,0,2.574181,0.191402,5.36846,0.238641,0.58


In [4]:
df_nea = pd.read_json(path_or_buf="file://localhost/Users/dtraverso/projects/nasa_nea/data/nea_extended.json")

In [5]:
df_nea.head()

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Ref,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms
0,1.782516,NaN,1893-2014,MPCLINUX,NaN,2457400.5,0.46,11.16,1804,2014-12-22,...,MPO355095,0.692794,2.315305,2.457230e+06,0,1.457878,0.222679,10.82861,0.559915,0.66
1,4.083845,NaN,1911-2016,MPCLINUX,NaN,2457400.5,0.15,15.50,1804,2016-02-06,...,MPO365250,0.917080,1.305524,2.456825e+06,0,2.633253,0.550874,11.56459,0.230656,0.42
2,3.884757,NaN,1918-2015,MPCLINUX,NaN,2457400.5,-0.12,13.40,1804,2015-07-06,...,MPO365253,0.840627,1.344521,2.457835e+06,0,2.478760,0.567217,9.35748,0.252553,0.73
3,4.085337,NaN,1924-2015,MPCLINUX,NaN,2457400.5,0.30,9.45,1804,2015-11-27,...,MPO370975,0.951983,1.298833,2.457392e+06,0,2.663276,0.533951,26.69239,0.226767,0.46
4,2.755192,NaN,1932-2015,MPCW,NaN,2457400.5,0.15,17.70,1804,2015-09-18,...,MPO347525,0.777716,1.602716,2.456954e+06,0,1.919387,0.435455,11.87965,0.370647,1.25


In [6]:
# Join MP and NEA dataframes to prepare for using drop duplicates function, which will remove all row instances 
# that have duplicates. This will provide us with an MP set without any NEA records in it.
df_mp_dups = pd.concat([df_mp, df_nea])

# Dropping duplicates based on columns that are highly probable to have unique values per-row
df_mp_pure = df_mp_dups.drop_duplicates(subset=['Peri', 'n'], keep=False)

In [7]:
# Creating the PHA set from all rows in NEA set that have the PHA flag set to true (1)
df_pha = df_nea[df_nea['PHA_flag'] == 1]

In [8]:
# Purifying the NEA set to remove any PHA records -- non-PHA records do not have the PHA flag field
df_nea_pure = df_nea[df_nea['PHA_flag'].isnull()]

In [9]:
print 'Master Set Count =', len(df_mp_pure.index)
print 'NEA Set Count =', len(df_nea_pure.index)
print 'PHA Set Count =', len(df_pha.index)

Master Set Count = 699486
NEA Set Count = 12418
PHA Set Count = 1679


In [10]:
# Adding Labels to the data, 2 = MP, 1 = NEA, 0 = PHA
df_mp_pure['Label'] = 2
df_nea_pure['Label'] = 1
df_pha['Label'] = 0

/Users/dtraverso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/dtraverso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/dtraverso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [11]:
# Join the purified sets together in preparation for feeding this cumulative set to classifiers for training & testing.
df_joined = pd.concat([df_mp_pure, df_nea_pure, df_pha])

In [12]:
df_joined.head()

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms,Label
0,2.977808,NaN,1801-2015,MPCLINUX,NaN,2457400.5,0.12,3.34,0000,2015-11-28,...,1.376113,1.277355,2.458235e+06,0,2.768112,0.075754,10.59166,0.214007,0.60,2
1,3.412843,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.11,4.13,0000,2015-10-16,...,1.312198,1.276541,2.456634e+06,0,2.772362,0.231024,34.84095,0.213515,0.58,2
2,3.353932,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.32,5.33,0000,2015-12-18,...,1.247217,1.297421,2.456854e+06,0,2.669763,0.256266,12.98987,0.225941,0.60,2
3,2.571675,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.32,3.20,0000,2015-12-15,...,1.171498,1.380322,2.456924e+06,0,2.361670,0.088922,7.14005,0.271566,0.60,2
4,3.066885,NaN,1845-2015,MPCLINUX,NaN,2457400.5,0.15,6.85,0000,2015-12-20,...,1.239938,1.319481,2.457419e+06,0,2.574181,0.191402,5.36846,0.238641,0.58,2


In [13]:
# Deleting unrelevant features from the dataset. Many of these columns are of 'String' or 'Object' type,
# and we are intentionally narrowing the set to float64 type fields for numerical analysis.
df_joined = df_joined.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
                            'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U', 'Arc_length'],axis=1)       

# Deprecated removal of additional fields for testing / debugging, ignore!
# df_joined = df_joined.copy().drop(['Num_obs', 'G',
#                                   'Critical_list_numbered_object_flag',  'Node', 'Peri', 'NEO_flag', 
#                                   'Num_opps', 'One_km_NEO_flag', 'One_opposition_object_flag', 'Orbital_period',
#                                   'PHA_flag', 'Semilatus_rectum', 'Synodic_period', 'rms',  'Tp','H'],
#                           axis=1)

df_joined = df_joined.copy().drop([ 'NEO_flag','PHA_flag'],
                          axis=1)

# Substitute null values with 0s, or most scikit-learn classifiers will throw errors
df_joined = df_joined.fillna(0.0)
# Print the remaining features in the massaged dataset.
df_joined.dtypes

Aphelion_dist                         float64
Critical_list_numbered_object_flag    float64
Epoch                                 float64
G                                     float64
H                                     float64
M                                     float64
Node                                  float64
Num_obs                               float64
Num_opps                                int64
One_km_NEO_flag                       float64
One_opposition_object_flag            float64
Orbital_period                        float64
Peri                                  float64
Perihelion_dist                       float64
Semilatus_rectum                      float64
Synodic_period                        float64
Tp                                    float64
a                                     float64
e                                     float64
i                                     float64
n                                     float64
rms                               

In [14]:
# Extract the Label column as a Pandas Series from the master set. This will be used for training / testing as 
# our Y.
sr_label = df_joined['Label']

# Drop the Label column from the master data set now that we have isolated it as a Series
df_joined = df_joined.drop('Label', axis=1)

# List current cumulative set columns to confirm correctness.
df_joined.columns

Index([u'Aphelion_dist', u'Critical_list_numbered_object_flag', u'Epoch', u'G',
       u'H', u'M', u'Node', u'Num_obs', u'Num_opps', u'One_km_NEO_flag',
       u'One_opposition_object_flag', u'Orbital_period', u'Peri',
       u'Perihelion_dist', u'Semilatus_rectum', u'Synodic_period', u'Tp', u'a',
       u'e', u'i', u'n', u'rms'],
      dtype='object')

In [15]:
# Make train and test sets, using cumulative DataFrame and Labels Series. Setting test data proportion to 
# 25% of total.
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_joined, sr_label, test_size=.25, random_state=0)

In [16]:
# Print the shapes of the training & test data to ensure all sets are in the expected dimensions.
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(535187, 22)
(178396, 22)
(535187,)
(178396,)


In [17]:
# We've commented out the SVM classifier as it was taking an unreasonably long processing time (hours...)
# clf = svm.SVC(kernel='linear', C=1)
# scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=3, scoring="f1_weighted")
# fitted = clf.fit(X_train, y_train)
# clf.score(X_test, y_test)
# print scores

In [18]:
#-----  Linear Discriminant Analysis w/ SVD
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Linear Discriminant Analysis --- Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Linear Discriminant Analysis --- Normalized confusion matrix')
print(cm_normalized)

/Users/dtraverso/anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Linear Discriminant Analysis --- Confusion matrix, without normalization
[[   234    155     22]
 [   731   2109    285]
 [   136    194 174530]]
Linear Discriminant Analysis --- Normalized confusion matrix
[[  5.69e-01   3.77e-01   5.35e-02]
 [  2.34e-01   6.75e-01   9.12e-02]
 [  7.78e-04   1.11e-03   9.98e-01]]


In [19]:
print('Linear Discriminant Analysis Score')
clf.score(X_test, y_test)

Linear Discriminant Analysis Score


0.99146281306755757

In [20]:
#-----  LogisticRegression

lr = linear_model.LogisticRegression(penalty='l1',dual=False,max_iter =100 )
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

y_pred = lr.predict(X_test)

In [21]:
print('Logistic Regression Score')
lr.score(X_test,y_test)

Logistic Regression Score


0.9962835489584968

In [22]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Logistic Regression --- Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Logistic Regression --- Normalized confusion matrix')
print(cm_normalized)


Logistic Regression --- Confusion matrix, without normalization
[[    16    395      0]
 [    18   2888    219]
 [     0     31 174829]]
Logistic Regression --- Normalized confusion matrix
[[  3.89e-02   9.61e-01   0.00e+00]
 [  5.76e-03   9.24e-01   7.01e-02]
 [  0.00e+00   1.77e-04   1.00e+00]]


In [23]:
#-----  Random Forest Classifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)

y_pred = rfc.predict(X_test)
print('Random FOrest Classifier Score')
rfc.score(X_test,y_test)


Random FOrest Classifier Score


0.99842485257516989

In [24]:
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Random FOrest Classifier --- Confusion matrix, without normalization')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Random FOrest Classifier --- Normalized confusion matrix')
print(cm_normalized)


Random FOrest Classifier --- Confusion matrix, without normalization
[[   182    229      0]
 [    51   3073      1]
 [     0      0 174860]]
Random FOrest Classifier --- Normalized confusion matrix
[[  4.43e-01   5.57e-01   0.00e+00]
 [  1.63e-02   9.83e-01   3.20e-04]
 [  0.00e+00   0.00e+00   1.00e+00]]
